In [28]:
"""
Author: Jiacheng Xie
Date: 2022/8/17
"""
from tkinter import Tk,filedialog, messagebox, Button, W,Label,Entry
import pyodbc
from pymssql import connect as connection
from pandas import read_xml, DataFrame,read_sql,concat,read_csv
from tqdm import tqdm
from sqlalchemy import create_engine
from time import time
from os import remove
import gc

# import the following to avoid no 'pymssql' found issue when packing
from pymssql import _mssql
from pymssql import _pymssql
import uuid
import decimal


global authenticationMode
authenticationMode = 0
global files
files = ()

"""
Find column names in the xml field of xevent files
"""
def findColumns(xml_pretty_str):
    return ' '.join(read_xml(xml_pretty_str)['name'].tolist())

"""
Visualize pd.to_sql, add a process bar
"""
def chunker(seq, size):
    # from http://stackoverflow.com/a/434328
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def insert_with_progress(df,conn, tableName):
    #conn = create_engine(DB_URI2, execution_options=dict(stream_results=True),fast_executemany=True)
    
    chunksize = int(len(df) / 10) # 10%
    if chunksize == 0:
        chunksize = 1

    with tqdm(total=len(df)) as pbar:
        for i, cdf in enumerate(chunker(df, chunksize)):
            replace = "replace" if i == 0 else "append"
            cdf.to_sql(con=conn, name=tableName, if_exists=replace, index=False)
            pbar.update(chunksize)

"""
This is the core function of the Xevent reader. 
It parses xml and edits the result to get correct values for each column.
Return a comma separated string of all column values.

"""            
def xmlToString2(xml_pretty_str, column_list_ordered):
    
    df_xml = read_xml(xml_pretty_str)#.set_index('name')

    names = list(df_xml['name'].values)
    
    # chain of if...else to deal with columns with xml data
    if 'input_relation' in names:

        pattern = '<data name="input_relation">'
        data = xml_pretty_str[xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'):xml_pretty_str.find('</value>', xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'))]
        df_xml.loc[df_xml['name'] == 'input_relation','value'] = data

    elif 'calculator' in names:

        pattern = '<data name="calculator">'
        data = xml_pretty_str[xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'):xml_pretty_str.find('</value>', xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'))]
        df_xml.loc[df_xml['name'] == 'calculator','value'] = data

    elif 'stats_collection' in names:

        pattern = '<data name="stats_collection">'
        data = xml_pretty_str[xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'):xml_pretty_str.find('</value>', xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'))]
        df_xml.loc[df_xml['name'] == 'stats_collection','value'] = data

    elif 'showplan_xml' in names:

        pattern = '<data name="showplan_xml">'
        data = xml_pretty_str[xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'):xml_pretty_str.find('</value>', xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'))]
        df_xml.loc[df_xml['name'] == 'showplan_xml','value'] = data

    elif 'output_parameters' in names:

        pattern = '<data name="output_parameters">'
        data = xml_pretty_str[xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'):xml_pretty_str.find('</value>', xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'))]
        df_xml.loc[df_xml['name'] == 'output_parameters','value'] = data

    elif 'data' in names:

        pattern = '<data name="data">'
        data = xml_pretty_str[xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'):xml_pretty_str.find('</value>', xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'))]
        df_xml.loc[df_xml['name'] == 'data','value'] = data

    elif 'execution_stats_report' in names:

        pattern = '<data name="execution_stats_report">'
        data = xml_pretty_str[xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'):xml_pretty_str.find('</value>', xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'))]
        df_xml.loc[df_xml['name'] == 'execution_stats_report','value'] = data

    elif 'xml_report' in names:

        pattern = '<data name="xml_report">'
        data = xml_pretty_str[xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'):xml_pretty_str.find('</value>', xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'))]
        df_xml.loc[df_xml['name'] == 'xml_report','value'] = data

    elif 'xml_report_filtered' in names:

        pattern = '<data name="xml_report_filtered">'
        data = xml_pretty_str[xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'):xml_pretty_str.find('</value>', xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'))]
        df_xml.loc[df_xml['name'] == 'xml_report_filtered','value'] = data

    elif 'server_memory_grants' in names:

        pattern = '<data name="server_memory_grants">'
        data = xml_pretty_str[xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'):xml_pretty_str.find('</value>', xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'))]
        df_xml.loc[df_xml['name'] == 'server_memory_grants','value'] = data

    elif 'blocked_process' in names:

        pattern = '<data name="blocked_process">'
        data = xml_pretty_str[xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'):xml_pretty_str.find('</value>', xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'))]
        df_xml.loc[df_xml['name'] == 'blocked_process','value'] = data

    else:

        pattern = '<data name="blocked_process_filtered">'
        data = xml_pretty_str[xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'):xml_pretty_str.find('</value>', xml_pretty_str.find('<value>', xml_pretty_str.find(pattern)+len(pattern))+len('<value>'))]
        df_xml.loc[df_xml['name'] == 'blocked_process_filtered','value'] = data
    # exclude
    if 'data_stream' in names:
        df_xml.drop(df_xml[df_xml['name'] == 'data_stream'].index, inplace = True)
        names.remove('data_stream')
    
    if 'timestamp' in names:
        df_xml.drop(df_xml[df_xml['name'] == 'timestamp'].index, inplace = True)
        names.remove('timestamp')

    if 'text' in df_xml:     
        df_xml['value'] = df_xml['text'].fillna(df_xml['value'])

    #df_xml['value'] = df_xml['value'].apply(lambda x: str(x).replace(',','^'))


    values = list(df_xml['value'].values)


    result = []

    for name in column_list_ordered:
        if name in names:
            result.append(str(values[names.index(name)]).replace(',','^'))
        else:
            result.append('None')
 
    return ','.join(result).replace('\t','').replace('\n','')


def setWinAuthentication():
    global authenticationMode
    authenticationMode = 0
    print('Windows Authentication Selected')
    print('Specify HOSTNAME, DATABASE, TABLE NAME:')

def setSQLAuthentication():
    global authenticationMode
    authenticationMode = 1
    print("SQL Authentication Selected")
    print('Specify HOSTNAME, DATABASE, TABLE NAME, USERNAME, PASSWORD')
    
def selectFiles():
    global files
    files = filedialog.askopenfilenames(filetypes=[('xevent files', '.xel'),('csv files', '.csv')])
    print('Files selected:')
    print(files)
    print('Select Authentication Mode:')

"""
The key idea is to save execution result into a csv file and then remove the "" to simulate the
process of creating columns. Since the editted file has the same format as a csv file, when reading
it again into a pandas dataframe would result in an expanded table with all columns of xevent names.
"""
def run():
    if len(files) == 0:
        messagebox.askokcancel(title = 'Message',message='No file selected!')
        return 'No file selected!'
    
    HOSTNAME = entryHOSTNAME.get()
    DATABASE = entryDATABASE.get()
    TableName = entryTableName.get()
    USERNAME = entryUSERNAME.get()
    PASSWORD = entryPASSWORD.get()
    if authenticationMode == 1 and (len(USERNAME) == 0 or len(PASSWORD) == 0):
        messagebox.askokcancel(title = 'Message',message='Please specify USERNAME and PASSWORD for SQL Authentication')
        return 'SQL Authentication Failed'
    if len(HOSTNAME) == 0:
        messagebox.askokcancel(title = 'Message',message='Please specify HOSTNAME')
        return 'Invalid input'
    if len(DATABASE) == 0:
        messagebox.askokcancel(title = 'Message',message='Please specify DATABASE')
        return 'Invalid input'
    if len(TableName) == 0:
        messagebox.askokcancel(title = 'Message',message='Please specify Table Name')
        return 'Invalid input'    
    
    
    # First connection, connect to master by default
    if authenticationMode == 0:
        print('Action: Connecting to ' + 'master' + " <Windows Authentication> ...")
    else:
        print('Action: Connecting to ' + 'master' + " <SQL Authentication>")
    DATABASE_master = 'master'
    DRIVER = 'ODBC+Driver+17+for+SQL+Server'
    DB_URI_SQL = 'mssql+pyodbc://{}:{}@{}/{}?driver={}'.format(
        USERNAME, PASSWORD, HOSTNAME, DATABASE_master,DRIVER
    )
    DB_URI_Win = 'mssql+pyodbc://{}/{}?trusted_connection=yes&driver={}'.format(
        HOSTNAME, DATABASE_master,DRIVER
    )

    DB_URI2_Win = 'mssql+pyodbc://{}/{}?trusted_connection=yes&driver={}'.format(
        HOSTNAME, DATABASE,DRIVER
    )
    DB_URI2_SQL = 'mssql+pyodbc://{}:{}@{}/{}?driver={}'.format(
        USERNAME, PASSWORD, HOSTNAME, DATABASE,DRIVER
    )

    if authenticationMode == 0:
        DB_URI = DB_URI_Win
        DB_URI2 = DB_URI2_Win
        conn = create_engine(DB_URI, execution_options=dict(stream_results=True),fast_executemany=True)
        connect = connection(host=HOSTNAME,
                                  database=DATABASE_master,
                                  autocommit=True)    
    
    else:
        DB_URI = DB_URI_SQL
        DB_URI2 = DB_URI2_SQL
        conn = create_engine(DB_URI, execution_options=dict(stream_results=True),fast_executemany=True)
        connect = connection(host=HOSTNAME,
                                  database=DATABASE_master,
                                  user=USERNAME,
                                  password=PASSWORD,
                                  autocommit=True)
    cursor = connect.cursor()
    print("... Connected ...")

    # create new database if not exist
    new_database = str("'" + DATABASE + "'")

    cursor.execute("select * From master.dbo.sysdatabases where name=" + new_database)
    DBresult = cursor.fetchall()

    if len(DBresult) == 0: # if DB not exist
        cursor.execute("CREATE DATABASE " + new_database.strip("'"))
        print('Message: ' + new_database.strip("'") + " created")
    else:
        print('Message: ' + new_database.strip("'") + " exist")
    connect.close() 

    # set table name
    print('Message: result will be written into '+ TableName + ' ...')
    print('Message: table with the same name will be repalced ...')

    # second connection, same Authentication as first
    if authenticationMode == 0:
        print('Action: Connecting to ' + new_database.strip("'") + " <Windows Authentication> ...")
    else:
        print('Action: Connecting to ' + 'master' + " <SQL Authentication>")

    conn = create_engine(DB_URI2, execution_options=dict(stream_results=True),fast_executemany=True)

    print("... Connected ...")
    # read from sql
    print('Action: Read files ...')

    df = DataFrame()
    for file in tqdm(files):
        file = str("'" + file + "'").replace('/',"\\")                                             
        query = "select * from sys.fn_xe_file_target_read_file(" +  file + ",null, null, null)"
        temp_df = read_sql(query, conn)
        df = concat([df,temp_df],ignore_index=True)
    print("Message:")
    print(df.info())
    df.rename(columns={'object_name':'event_name'}, inplace = True)

    df_col = df.drop_duplicates(['event_name']).copy()
    df_col['columns'] = df_col['event_data'].apply(findColumns)
    column_list_string = df_col['columns'].values.tolist()
    column_list_set = set(' '.join(column_list_string).split())
    column_list_set.discard('timestamp')
    #column_list_set.discard('object_name')
    column_list_set.discard('data_stream')
    column_list_ordered = list(column_list_set)
    column_list_ordered.sort()
    column_list_string = str(column_list_ordered).replace('[','').replace(']','').replace("'",'').replace(" ",'')
    print('Message: the following columns will be written ...')
    print(column_list_ordered)

    # Parsing
    tqdm.pandas(desc='pandas bar')
    print("Action: Parsing xevent ...")
    df[column_list_string] = df['event_data'].progress_apply(xmlToString2,column_list_ordered=column_list_ordered)

    df2 = DataFrame(df, columns = ['event_name','timestamp_utc',column_list_string])
    
    del df
    gc.collect()

    # save result to csv
    print('Action: Save and edit temp csv file ...')
    temp_name = str(int(time()))
    df2.to_csv('./'+ temp_name + '.csv',index = False)
    
    del df2
    gc.collect()

    with open('./'+ temp_name + '.csv','r',encoding="utf-8") as f:
        lines = f.readlines()
    with open('./'+ temp_name + '.csv', 'w',encoding="utf-8") as f_w:
        for line in tqdm(lines):
            line = line.replace('"','')#.replace('[','').replace(']','')
            f_w.write(line)

    # read from editted csv file
    print("Action: Reading csv file ...")
    chunks2 = read_csv('./'+ temp_name + '.csv',dtype=str,chunksize=2000)

    df_done = DataFrame()
    for chunk in tqdm(chunks2):
        df_done = concat([df_done, chunk],ignore_index=True)
    print('Message: ')
    print(df_done.info())

    # importing to SQL Server

    print('Action: Importing to SQL Server table ...')
    insert_with_progress(df_done,conn, TableName)
    print('Job done!')
    
    del df_done
    gc.collect()
    
    delTemp = messagebox.askyesno(title = 'Message',message='Job Done! Delete temp csv file?')
    if delTemp:
        remove('./'+ temp_name + '.csv')
    print("Press Enter to Continue ...")
    
    
"""
Importing csv files into SQL Server.
"""    
def run_csv():
    if len(files) == 0:
        messagebox.askokcancel(title = 'Message',message='No file selected!')
        return 'No file selected!'
    
    HOSTNAME = entryHOSTNAME.get()
    DATABASE = entryDATABASE.get()
    TableName = entryTableName.get()
    USERNAME = entryUSERNAME.get()
    PASSWORD = entryPASSWORD.get()
    if authenticationMode == 1 and (len(USERNAME) == 0 or len(PASSWORD) == 0):
        messagebox.askokcancel(title = 'Message',message='Please specify USERNAME and PASSWORD for SQL Authentication')
        return 'SQL Authentication Failed'
    if len(HOSTNAME) == 0:
        messagebox.askokcancel(title = 'Message',message='Please specify HOSTNAME')
        return 'Invalid input'
    if len(DATABASE) == 0:
        messagebox.askokcancel(title = 'Message',message='Please specify DATABASE')
        return 'Invalid input'
    if len(TableName) == 0:
        messagebox.askokcancel(title = 'Message',message='Please specify Table Name')
        return 'Invalid input'    
    
    
    # First connection, connect to master by default
    if authenticationMode == 0:
        print('Action: Connecting to ' + 'master' + " <Windows Authentication> ...")
    else:
        print('Action: Connecting to ' + 'master' + " <SQL Authentication>")
    DATABASE_master = 'master'
    DRIVER = 'ODBC+Driver+17+for+SQL+Server'
    DB_URI_SQL = 'mssql+pyodbc://{}:{}@{}/{}?driver={}'.format(
        USERNAME, PASSWORD, HOSTNAME, DATABASE_master,DRIVER
    )
    DB_URI_Win = 'mssql+pyodbc://{}/{}?trusted_connection=yes&driver={}'.format(
        HOSTNAME, DATABASE_master,DRIVER
    )

    DB_URI2_Win = 'mssql+pyodbc://{}/{}?trusted_connection=yes&driver={}'.format(
        HOSTNAME, DATABASE,DRIVER
    )
    DB_URI2_SQL = 'mssql+pyodbc://{}:{}@{}/{}?driver={}'.format(
        USERNAME, PASSWORD, HOSTNAME, DATABASE,DRIVER
    )

    if authenticationMode == 0:
        DB_URI = DB_URI_Win
        DB_URI2 = DB_URI2_Win
        conn = create_engine(DB_URI, execution_options=dict(stream_results=True),fast_executemany=True)
        connect = connection(host=HOSTNAME,
                                  database=DATABASE_master,
                                  autocommit=True)    
    
    else:
        DB_URI = DB_URI_SQL
        DB_URI2 = DB_URI2_SQL
        conn = create_engine(DB_URI, execution_options=dict(stream_results=True),fast_executemany=True)
        connect = connection(host=HOSTNAME,
                                  database=DATABASE_master,
                                  user=USERNAME,
                                  password=PASSWORD,
                                  autocommit=True)
    cursor = connect.cursor()
    print("... Connected ...")

    # create new database if not exist
    new_database = str("'" + DATABASE + "'")

    cursor.execute("select * From master.dbo.sysdatabases where name=" + new_database)
    DBresult = cursor.fetchall()

    if len(DBresult) == 0: # if DB not exist
        cursor.execute("CREATE DATABASE " + new_database.strip("'"))
        print('Message: ' + new_database.strip("'") + " created")
    else:
        print('Message: ' + new_database.strip("'") + " exist")
    connect.close() 

    # set table name
    print('Message: result will be written into '+ TableName + ' ...')
    print('Message: table with the same name will be repalced ...')

    # second connection, same Authentication as first
    if authenticationMode == 0:
        print('Action: Connecting to ' + new_database.strip("'") + " <Windows Authentication> ...")
    else:
        print('Action: Connecting to ' + 'master' + " <SQL Authentication>")

    conn = create_engine(DB_URI2, execution_options=dict(stream_results=True),fast_executemany=True)

    print("... Connected ...")
    
    # read from editted csv file
    print("Action: Reading csv file ...")
    chunks2 = read_csv(files[0],dtype=str,chunksize=2000)

    df_done = DataFrame()
    for chunk in tqdm(chunks2):
        df_done = concat([df_done, chunk],ignore_index=True)
    print('Message: ')
    print(df_done.info())

    # importing to SQL Server

    print('Action: Importing to SQL Server table ...')
    insert_with_progress(df_done,conn, TableName)
    print('Job done!')
    
    del df_done
    gc.collect()
    
    delTemp = messagebox.askyesno(title = 'Message',message='Job Done! Delete temp csv file?')
    if delTemp:
        remove(files[0])
    print("Press Enter to Continue ...")
    
print('Select Xevent Files:')
myWindow = Tk()

myWindow.title('Xevent Reader')
myWindow.resizable(0, 0)

b0=Button(myWindow, text='Select Files', relief='ridge', width=45, height=2, command = selectFiles)
b0.grid(row=0, columnspan=2, sticky=W, padx=5,pady=5)

b1=Button(myWindow, text='SQL Authentication', relief='ridge', width=20, height=2, command = setSQLAuthentication)
b1.grid(row=1, column=0, sticky=W, padx=5,pady=5)
b2=Button(myWindow, text='Windows Authentication',relief='ridge',width=20, height=2, command = setWinAuthentication)
b2.grid(row=1, column=1, sticky=W, padx=5, pady=5)

    
Label(myWindow, text="HOSTNAME").grid(row=2)
Label(myWindow, text="DATABASE").grid(row=3)
Label(myWindow, text="TABLE NAME").grid(row=4)
Label(myWindow, text="Skip Following Blanks If Windows Authentication Is Selected",relief = 'groove',bd = 4).grid(row=5,columnspan=2)
Label(myWindow, text="USERNAME").grid(row=6)
Label(myWindow, text="PASSWORD").grid(row=7)

entryHOSTNAME=Entry(myWindow)
entryDATABASE=Entry(myWindow)
entryUSERNAME=Entry(myWindow)
entryPASSWORD=Entry(myWindow)
entryTableName=Entry(myWindow)
entryHOSTNAME.grid(row=2, column=1)
entryDATABASE.grid(row=3, column=1)
entryTableName.grid(row=4, column=1)
entryUSERNAME.grid(row=6, column=1)
entryPASSWORD.grid(row=7, column=1)


b3=Button(myWindow, text='Run: full execution',relief='ridge',width=45, height=2, command = run)
b3.grid(row=8, columnspan=2, sticky=W, padx=5, pady=5)

b4=Button(myWindow, text='Run: import csv to SQL only',relief='ridge',width=45, height=2, command = run_csv)
b4.grid(row=9, columnspan=2, sticky=W, padx=5, pady=5)


myWindow.mainloop()



Select Xevent Files:
Files selected:

Select Authentication Mode:


In [25]:
files

()